In [1]:
import pandas as pd
!pip install transformers

     |████████████████████████████████| 4.2 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 70.0 MB/s 
     |████████████████████████████████| 6.6 MB 41.7 MB/s 
     |████████████████████████████████| 84 kB 4.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [23]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [24]:
link = 'https://drive.google.com/file/d/1Cus0W0LLYX7nhl6XLV50iINRh72cRCEv/view?usp=sharing' 

In [25]:
# to get the id part of the file
id = link.split("/")[-2]
 
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Survey_Results_Labeled_Feb.csv') 
 
df = pd.read_csv('Survey_Results_Labeled_Feb.csv')
df

,Unnamed: 0,text,1,2,3,Label
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral
1,Q2,@KingJames You have been awfully and painfully...,0.0,0.0,1.0,negative
2,Q3,"""@PrillLee @MarthaP29033882 @DanRather It is n...",0.0,0.3,0.7,negative
3,Q4,@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,0.3,0.5,0.2,neutral
4,Q5,"@IAmMrKobayashi World War III; Full video, How...",0.2,0.7,0.1,neutral
...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative
98,Q97,"""@EP_President @Europarl_EN @EU_Commission STO...",0.0,0.3,0.7,negative
99,Q98,"""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...",0.0,0.1,0.9,negative
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative


In [26]:
df['Sentiment'] = 0

In [27]:
df

,Unnamed: 0,text,1,2,3,Label,Sentiment
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral,0
1,Q2,@KingJames You have been awfully and painfully...,0.0,0.0,1.0,negative,0
2,Q3,"""@PrillLee @MarthaP29033882 @DanRather It is n...",0.0,0.3,0.7,negative,0
3,Q4,@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,0.3,0.5,0.2,neutral,0
4,Q5,"@IAmMrKobayashi World War III; Full video, How...",0.2,0.7,0.1,neutral,0
...,...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative,0
98,Q97,"""@EP_President @Europarl_EN @EU_Commission STO...",0.0,0.3,0.7,negative,0
99,Q98,"""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...",0.0,0.1,0.9,negative,0
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative,0


In [28]:
from google.colab import drive
drive.mount('drive')
import sys
sys.path.append('drive/My Drive/Colab Notebooks')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [29]:
#code from NLPiation

import torch
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Load the BERT Tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-large-uncased')

# The dataset class
class TheDataset(torch.utils.data.Dataset):

    def __init__(self, reviews, sentiments, tokenizer):
        self.reviews    = reviews
        self.sentiments = sentiments
        self.tokenizer  = tokenizer
        self.max_len    = tokenizer.model_max_length
  
    def __len__(self):
        return len(self.reviews)
  
    def __getitem__(self, index):
        review = str(self.reviews[index])
        sentiments = self.sentiments[index]

        encoded_review = self.tokenizer.encode_plus(
            review,
            add_special_tokens    = True,
            max_length            = 128,
            return_token_type_ids = False,
            return_attention_mask = True,
            return_tensors        = 'pt',
            padding               = "max_length",
            truncation            = True
        )

        return {
            'input_ids': encoded_review['input_ids'][0],
            'attention_mask': encoded_review['attention_mask'][0],
            'labels': torch.tensor(sentiments, dtype=torch.long)
        }

# Prepare the Train/Validation sets
#train_set_dataset = TheDataset(
    #reviews    = train_set.text.tolist(),
    #sentiments = train_set.Sentiment.tolist(),
    #tokenizer  = tokenizer,
#)

#valid_set_dataset = TheDataset(
    #reviews    = valid_set.text.tolist(),
    #sentiments = valid_set.Sentiment.tolist(),
    #tokenizer  = tokenizer,
#)

# Load the BERT model
#model = BertForSequenceClassification.from_pretrained("drive/My Drive/Colab Notebooks/epoch 4", problem_type="single_label_classification", num_labels=3)

# Freeze the whole BERT model and train just the classifier
#for name, param in model.bert.named_parameters():
    #param.requires_grad = False

# Freeze BERT except the pooler layer
#for name, param in model.bert.named_parameters():
    #if not name.startswith('pooler'):
        #param.requires_grad = False

# Freeze the first 18 layers of the BERT
#for name, param in model.bert.named_parameters():
    #if ( not name.startswith('pooler') ) and "layer.23" and "layer.22" and "layer.21" and "layer.20" and "layer.19" and "layer.18" not in name :
        #param.requires_grad = False

# The function to get the accuracy
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

# Define the training parameters
training_args = TrainingArguments(
    output_dir                  = 'drive/My Drive/Colab Notebooks',
    num_train_epochs            = 2,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 64,
    warmup_steps                = 10000,
    learning_rate               = 5e-5,
    weight_decay                = 0.01,
    adam_beta1                  = 0.9,
    adam_beta2                  = 0.999,
    save_strategy               = "epoch",
    evaluation_strategy         = "epoch",
    eval_steps                  = 1,
    #save_total_limit            = 5,
    #metric_for_best_model       = 'eval_loss',
    load_best_model_at_end      =True
)

# Define the Huggingface Trainer object
#trainer = Trainer(
    #model           = model,
    #args            = training_args,
    #train_dataset   = train_set_dataset,
    #eval_dataset    = valid_set_dataset,
    #compute_metrics = compute_metrics,
    #callbacks       = [EarlyStoppingCallback(early_stopping_patience=3)]
#)


https://huggingface.co/bert-large-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpoewh52vm


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/bert-large-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/300ecd79785b4602752c0085f8a89c3f0232ef367eda291c79a5600f3778b677.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/300ecd79785b4602752c0085f8a89c3f0232ef367eda291c79a5600f3778b677.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
https://huggingface.co/bert-large-uncased/resolve/main/config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp3flg88v3


Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

storing https://huggingface.co/bert-large-uncased/resolve/main/config.json in cache at /root/.cache/huggingface/transformers/1cf090f220f9674b67b3434decfe4d40a6532d7849653eac435ff94d31a4904c.1d03e5e4fa2db2532c517b2cd98290d8444b237619bd3d2039850a6d5e86473d
creating metadata file for /root/.cache/huggingface/transformers/1cf090f220f9674b67b3434decfe4d40a6532d7849653eac435ff94d31a4904c.1d03e5e4fa2db2532c517b2cd98290d8444b237619bd3d2039850a6d5e86473d
loading configuration file https://huggingface.co/bert-large-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/1cf090f220f9674b67b3434decfe4d40a6532d7849653eac435ff94d31a4904c.1d03e5e4fa2db2532c517b2cd98290d8444b237619bd3d2039850a6d5e86473d
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
 

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/bert-large-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpmqdbljw9


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-large-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://h

In [30]:
# Load the checkpoint
model = BertForSequenceClassification.from_pretrained("drive/My Drive/Colab Notebooks/BERT_Large/Bert_large_Feb/epoch 2", num_labels=3)

# Make the test set ready
test_set_dataset = TheDataset(
    reviews    = df.text.tolist(),
    sentiments = df.Sentiment.tolist(),
    tokenizer  = tokenizer,
)

training_args = TrainingArguments(
    output_dir = 'drive/My Drive/Colab Notebooks',
    do_predict = True
)

trainer = Trainer(
    model           = model,
    args            = training_args,
    compute_metrics =compute_metrics,
)

PredictionOutput = trainer.predict(test_set_dataset)

loading configuration file drive/My Drive/Colab Notebooks/BERT_Large/Bert_large_Feb/epoch 2/config.json
Model config BertConfig {
  "_name_or_path": "bert-large-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
import numpy as np
np.argmax(PredictionOutput[0][4])

0

In [32]:
PredictionOutput[0][4]

array([ 5.443303, -2.523331, -2.344538], dtype=float32)

In [33]:
df['Bert_Large label'] = [np.argmax(i) for i in (PredictionOutput[0])]

In [34]:
df['Bert_Large label'].replace({0: "negative", 1: "neutral", 2: 'positive'}, inplace=True)

In [35]:
results = df['Bert_Large label'].value_counts()

In [36]:
results

negative    69
positive    27
neutral      6
Name: Bert_Large label, dtype: int64

In [37]:
df

,Unnamed: 0,text,1,2,3,Label,Sentiment,Bert_Large label
0,Q1,"""Human rights groups have raised alarm over wh...",0.0,0.7,0.3,neutral,0,negative
1,Q2,@KingJames You have been awfully and painfully...,0.0,0.0,1.0,negative,0,negative
2,Q3,"""@PrillLee @MarthaP29033882 @DanRather It is n...",0.0,0.3,0.7,negative,0,negative
3,Q4,@AntoineSallesP @JoshuaHabib4 @JLMelenchon Why...,0.3,0.5,0.2,neutral,0,negative
4,Q5,"@IAmMrKobayashi World War III; Full video, How...",0.2,0.7,0.1,neutral,0,negative
...,...,...,...,...,...,...,...,...
97,Q96,There are 7 or 8 Billionaires in the UKRAINE! ...,0.4,0.0,0.6,negative,0,negative
98,Q97,"""@EP_President @Europarl_EN @EU_Commission STO...",0.0,0.3,0.7,negative,0,negative
99,Q98,"""@vonderleyen STOP RUSSIAN AGGRESSION AGAINST ...",0.0,0.1,0.9,negative,0,negative
100,Q99,"""""However difficult it may be, our pain will b...",0.4,0.1,0.5,negative,0,negative


In [38]:
#delete control tweets from set
df = df.drop(33)
df = df.drop(67)

In [39]:
#change column names to make clear what is what
df = df.rename(columns={"Label":"Manual Label"})

In [40]:
#drop unnecessary column used as fake true label.
df = df.drop('Sentiment',axis=1)

In [41]:
 import sklearn
 from sklearn.metrics import balanced_accuracy_score, precision_recall_fscore_support
 b_a_Bert_Large = balanced_accuracy_score(df['Manual Label'], df['Bert_Large label'])
 p_r_f1_class_Bert_Large = precision_recall_fscore_support(df['Manual Label'], df['Bert_Large label'], average=None, labels=['negative', 'neutral', 'positive'])
 p_r_f1_Bert_Large = precision_recall_fscore_support(df['Manual Label'], df['Bert_Large label'], average='macro')

In [42]:
print(b_a_Bert_Large)
print(p_r_f1_Bert_Large)
print(p_r_f1_class_Bert_Large)

0.4209486166007905
(0.3369783810960281, 0.4209486166007905, 0.31224171539961015, None)
(array([0.48529412, 0.33333333, 0.19230769]), array([0.7173913 , 0.04545455, 0.5       ]), array([0.57894737, 0.08      , 0.27777778]), array([46, 44, 10]))


In [ ]:
import pandas as pd
y_actu = df['Manual Label']
y_pred = df['Bert_Large label']
df_confusion = pd.crosstab(y_actu, y_pred)

In [ ]:
df_confusion

Bert_Large label,negative,neutral,positive
Manual Label,,,
negative,33,1,12
neutral,33,2,9
positive,2,3,5


In [ ]:
#Get accuracies per class order (negative, neutral, positive)
from sklearn.metrics import confusion_matrix
y_true = df['Manual Label']
y_pred = df['Bert_Large label']
matrix = confusion_matrix(y_true, y_pred)
matrix.diagonal()/matrix.sum(axis=1)

array([0.7173913 , 0.04545455, 0.5       ])

In [ ]:
from google.colab import drive
drive.mount('drive')

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [ ]:
df.to_csv('BERT_Large_Test_Dec.csv')
!cp BERT_Large_Test_Dec.csv "/content/drive/My Drive/Colab Notebooks"

